# Learning From Data Final

https://work.caltech.edu/homework/final.pdf

Python imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Nonlinear transforms

# 1

$\mathcal X \in \mathbb R^2$

$z$ is a linear combination of $(1, x_1, x_2, x_1x_2, x_1^2, x_2^2, \cdots, x_1^Q, x_2^Q)$.

In [ ]:
from scipy.special import binom
# We have 1, x_1, x_2.
# How many ways are there to choose Q items?

Q = 10
total = 0
for a in range(0, Q+1):
    for b in range(0, Q+1):
        for c in range(0, Q+1):
            if b == 0 and c == 0:
                continue
            if a+b+c == Q:
                total += 1
                
total

In [ ]:
total = 0
for a in range(0, Q+1):
    for b in range(0, Q+1):
        if a + b <= 10 and a + b > 0:
            total += 1
            
total

In [ ]:
import sympy as sp
sp.expand(sp.sympify("1 + x + y")**10).__str__().count("+")

According to "stars and bars", this can be interpeted as the number of permutations of the string `**********||`. I.e. number of different ways to divide 10 numbers in 3 categories.

We subtract 1 to account for the case where all the stars were put in the box corresponding to "1".


In [ ]:
Q = 10
d = 2
binom(Q + d, Q) - 1

**Answer: Alternative 1E, none of the above -- 65**

# Bias and Variance

# 2

$\bar g$ is expected value of $g^{(\mathcal D)}$ over all data sets $\mathcal D$. 

When can $\bar g \notin \mathcal H$?

a: All g's would be the same, and all of them in H.  
b: g would still be in H.  
c: g would still be in H.  
d: g would still be in H.  
e: Must be correct. It is nonsense that the expected value of something in H can be outside H.  

<p style="color:red;"><b>Answer: Alternative 2E, none of the above.</b></p>

---

Wrong. See explanation here: http://book.caltech.edu/bookforum/showthread.php?t=4624

The expected hypothesis is computed by taking an average of hypotheses. That is, e.g. (h1+h2)/2.

Let A be a bad nondeterministic algorithm that always returns either h1 or h2 with equal probability.

A logistic model is of the form $h(x) = \theta(w^T x)$ where $\theta(s) = \frac{e^s}{1+e^s} = \frac{1}{1 + e^{-s}}$

Can we think of a case where the sum of two logistic hypotheses $h_1$ and $h_2$ cannot be written as $\theta(s)$?

Indeed, imagine if h1 always has weights (0, 1) and h2 always has weights (0, -2).
The expected model does not have weights (0, -0.5). It is not even a logistic model. See  the following plot

In [ ]:

def theta(s):
    return 1 / (1 + np.exp(-s))

def h1(x0, x1):
    w0 = 0
    w1 = 1
    return theta(w0 * x0 + w1 * x1)

def h2(x0, x1):
    w0 = 0
    w1 = -2
    return theta(w0 * x0 + w1 * x1)

xx = np.linspace(-10, 10)
plt.plot(xx, h1(1, xx))
plt.plot(xx, h2(1, xx))
plt.plot(xx, 1/2 * (h1(1, xx) + h2(1, xx)))
plt.legend(["h1", "h2", "E[h]"])
plt.show()

$h_3(x)$ is clearly not a logistic function.

# Overfitting

# 3

Overfitting means fitting the data more than warranted. I.e. choosing a too complex model for the amount of data. Fitting to the noise.

a: If there is overfitting, there must be two or more hypotheses that have different values of E_in. -- True. We have chosen a too complex model that fits the input data too well, and in order to do that we had to compare multiple models.

b: If there is overfitting, there must be two or more hypotheses that have different values of E_out. -- True. If all hypotheses were equally good, we cannot really call this overfitting.

c: If there is overfitting, there must be two or more hypotheses that have different values of (E_out - E_in). -- ???

d: We can always determine if there is overfitting by comparing the values of E_out - E_in. -- ???

e: We cannot determine overfitting based on one hypothesis only. -- True.

# 4

Deterministic noise is the error due to complexity in the target function that our model cannot approximate.

Thus deterministic noise depends on our hypothesis set.

**Answer: Alternative 4D: Stochastic noise does not depend on the hypothesis set.**

# 5

The least squares linear regression solution satisfies the constraint. Thus the regularization has no effect, and w_reg = w_lin.

**Answer: Alternative A**

# 6

**Answer: Alternative B -- soft-order constraints can be translated into augmented error**

# 7

In [ ]:
data_train = np.loadtxt("features.train")
data_test = np.loadtxt("features.test")
y_train, X_train = data_train[:, :1], data_train[:, 1:]
y_test, X_test = data_test[:, :1], data_test[:, 1:]


Since I already implemented regularized least squares regression in the previous homework, I will use sklearn here.

> from sklearn.linear_model import Ridge  
Minimizes the objective function:  
||y - Xw||^2_2 + alpha * ||w||^2_2

In [ ]:
from sklearn.linear_model import RidgeClassifier

def get_n_vs_m_dataset(ns, ms, which):
    if which == "train":
        data = data_train.copy()
    elif which == "test":
        data = data_test.copy()
    else:
        1/0
    in_ns = np.isin(data[:,0], ns)
    in_ms = np.isin(data[:,0], ms)
    in_either = np.logical_or(in_ns, in_ms)
    data[in_ns,0] = +1.0
    data[in_ms,0] = -1.0
    X, y = data[in_either,1:], data[in_either,0]
    return np.hstack([np.ones([X.shape[0], 1]), X]), y
    
X, y = get_n_vs_m_dataset([5], [0, 1, 2, 3, 4, 6, 7, 8, 9], "train")


def plot(X, y):
    above = np.where(y > 0)
    below = np.where(y < 0)
    plt.scatter(X[below,1], X[below,2], s=1)
    plt.scatter(X[above,1], X[above,2], s=1)
    plt.legend(["ms", "ns"])
    plt.xlabel("Intensity")
    plt.ylabel("Symmetry")

In [ ]:
def plot_decision_boundary(clf):
    N = 1000
    X = np.zeros((N,3))
    X[:,0] = 1
    X[:,1] = 2*np.random.rand(N) - 1
    X[:,2] = np.random.rand(N)*(-14) + 7
    y = clf.predict(X)
    plot(X, y)

def in_sample_error(ns, ms):
    X, y = get_n_vs_m_dataset(ns, ms, "train")
    Z = X # = (1, x1, x2)
    clf = RidgeClassifier(alpha=1.0, fit_intercept=False)
    clf.fit(Z, y)
    plot_decision_boundary(clf)
    plot(X, y)
    plt.show()
    return np.mean(clf.predict(Z) != y)
    
print(in_sample_error([5], [0, 1, 2, 3, 4, 6, 7, 8, 9]))
print(in_sample_error([6], [0, 1, 2, 3, 4, 5, 7, 8, 9]))
print(in_sample_error([7], [0, 1, 2, 3, 4, 5, 6, 8, 9]))
print(in_sample_error([8], [0, 1, 2, 3, 4, 5, 6, 7, 9]))
print(in_sample_error([9], [0, 1, 2, 3, 4, 5, 6, 7, 8]))

All the classifiers are horrible, but 8 vs all has the lowest E_in.
This is since the training the estimator always guesses -1, and this one coincidentally has fewer of that that class compared to other numbers.

**Alternative 7D: 8 vs all**

# 8

In [ ]:
def plot_decision_boundary2(clf, phi):
    N = 1000
    X = np.zeros((N,3))
    X[:,0] = 1
    X[:,1] = 2*np.random.rand(N) - 1
    X[:,2] = np.random.rand(N)*(-14) + 7
    y = clf.predict(phi(X))
    plot(X, y)

def phi(X):
    Z = np.zeros((X.shape[0], 6))
    Z[:,0] = 1
    Z[:,1] = X[:,1]
    Z[:,2] = X[:,2]
    Z[:,3] = X[:,1]*X[:,2]
    Z[:,4] = X[:,1]**2
    Z[:,5] = X[:,2]**2
    return Z

def out_of_sample_error(ns, ms):
    X_train, y_train = get_n_vs_m_dataset(ns, ms, "train")
    X_test, y_test = get_n_vs_m_dataset(ns, ms, "test")
    
    Z_train = phi(X_train)
    Z_test = phi(X_test)

    clf = RidgeClassifier(alpha=1, fit_intercept=False)
    clf.fit(Z_train, y_train)
    plot_decision_boundary2(clf, phi)
    plot(X_train, y_train)
    plt.show()
    return np.mean(clf.predict(Z_test) != y_test)
    
print("0 vs all Eout", out_of_sample_error([0], [1, 2, 3, 4, 5, 6, 7, 8, 9]))
print("1 vs all Eout", out_of_sample_error([1], [0, 2, 3, 4, 5, 6, 7, 8, 9]))
print("2 vs all Eout", out_of_sample_error([2], [0, 1, 3, 4, 5, 6, 7, 8, 9]))
print("3 vs all Eout", out_of_sample_error([3], [0, 1, 2, 4, 5, 6, 7, 8, 9]))
print("4 vs all Eout", out_of_sample_error([4], [0, 1, 2, 3, 5, 6, 7, 8, 9]))

A little gotcha here: The intercept is not penalized for sklearn's Ridge method when using fit_intercept=True.
So instead add a ones column and set fit_intercept=False to get the desired behavior of also penalizing the intercept term w0.

**Alternative 8B: 1 vs all has the lowest E_out**

# 9

In [ ]:
def check_all(ns, ms):
    print("{} vs all".format(ns[0]))
    X_train, y_train = get_n_vs_m_dataset(ns, ms, "train")
    X_test, y_test = get_n_vs_m_dataset(ns, ms, "test")
    
    Z_train = phi(X_train)
    Z_test = phi(X_test)
    
    clf = RidgeClassifier(alpha=1, fit_intercept=False)

    # No transform E_in
    clf.fit(X_train, y_train)
    print("  No transform E_in\t", np.mean(clf.predict(X_train) != y_train))
    # No transform E_out
    print("  No transform E_out\t", np.mean(clf.predict(X_test) != y_test))
    # Transform E_in
    clf.fit(Z_train, y_train)
    print("     Transform E_in\t", np.mean(clf.predict(Z_train) != y_train))
    # Transform E_out
    print("     Transform E_out\t", np.mean(clf.predict(Z_test) != y_test))

check_all([1], [0, 2, 3, 4, 5, 6, 7, 8, 9])
check_all([2], [0, 1, 3, 4, 5, 6, 7, 8, 9])
check_all([3], [0, 1, 2, 4, 5, 6, 7, 8, 9])
check_all([4], [0, 1, 2, 3, 5, 6, 7, 8, 9])
check_all([5], [0, 1, 2, 3, 4, 6, 7, 8, 9])
check_all([6], [0, 1, 2, 3, 4, 5, 7, 8, 9])
check_all([7], [0, 1, 2, 3, 4, 5, 6, 8, 9])
check_all([8], [0, 1, 2, 3, 4, 5, 6, 7, 9])
check_all([9], [0, 1, 2, 3, 4, 5, 6, 7, 8])

a: False. Doesn't look like overfitting to me.  
b: False: No, transform doesn't improve E_out  
c: False: There is a slight difference in out of sample performance. E.g. 5 vs all is _slightly_ better.  
d: False: The transform sometimes sometimes has the same out of sample performance.  
e: True: The performance is improved, but only by a small percentage.

In [ ]:
five_vs_all_improvement_percentage = -(0.07922272047832586 - 0.07972097658196313)/0.07972097658196313
print(five_vs_all_improvement_percentage)

**Alternative 9E**


# 10

In [ ]:
ns = [5]
ms = [1]
print("1 vs 5")

X_train, y_train = get_n_vs_m_dataset(ns, ms, "train")
X_test, y_test = get_n_vs_m_dataset(ns, ms, "test")

Z_train = phi(X_train)
Z_test = phi(X_test)


#########################################################################

print("lambda=1")
clf = RidgeClassifier(alpha=1, fit_intercept=False)

# Transform E_in
clf.fit(Z_train, y_train)
print("     Transform E_in\t", np.mean(clf.predict(Z_train) != y_train))
# Transform E_out
print("     Transform E_out\t", np.mean(clf.predict(Z_test) != y_test))


plot_decision_boundary2(clf, phi)
plot(X_train, y_train)
plt.title("Training data")
plt.show()

plot_decision_boundary2(clf, phi)
plot(X_test, y_test)
plt.title("Test data")
plt.show()

##############################################################################

print("lambda=0.01")
clf2 = RidgeClassifier(alpha=0.01, fit_intercept=False)

# Transform E_in
clf2.fit(Z_train, y_train)
print("     Transform E_in\t", np.mean(clf2.predict(Z_train) != y_train))
# Transform E_out
print("     Transform E_out\t", np.mean(clf2.predict(Z_test) != y_test))

plot_decision_boundary2(clf2, phi)
plot(X_train, y_train)
plt.title("Training data")
plt.show()

plot_decision_boundary2(clf2, phi)
plot(X_test, y_test)
plt.title("Test data")
plt.show()

Another gotcha: RidgeRegression in sklearn doesn't work properly with solver="lsqr"?

Anyways looks like we have overfitting, since E_in goes slightly down and E_out goes slightly up.

**Alternative 10A: Overfitting occurs**